
# Mathematical Model of Dopamine D1 receptor - Workshop

In this tutorial, we will create and simulate a mathematical model of a signaling pathway of Dopamine D1 receptor. 

<br><br>

<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1743961378/Screenshot_2025-04-06_at_19.41.52_fzmfxu.png"  width=600 align='center' style="margin-top:30px"/>

<br><br>
This tutorial, can be completed in Google Colab or in your computer if python, juypyter notebook is installed. If you'd like to open this notebook in Colab, you can use the following link:

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/DopamineD1/blob/master/Notebook.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>

    
Reference: _*Nair, A. G. et al (2015). Sensing Positive versus Negative Reward Signals through Adenylyl Cyclase-Coupled GPCRs in Direct and Indirect Pathway Striatal Medium Spiny Neurons. The Journal of Neuroscience : the Official Journal of the Society for Neuroscience, 35(41), 14017–14030.*_ 


## Setting Google Collab

In [ ]:
%%capture
!pip install --quiet ssbtoolkit

In [ ]:
import os
os.kill(os.getpid(), 9)

## Setting environment

In [ ]:
from pysb import *
from pysb.simulator import ScipyOdeSimulator
from pysb.core import SelfExporter

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'


import platform, site, os
distpath = site.getsitepackages()[0]
if platform.system() == 'Linux':
    BioNetGen=os.path.join(distpath, 'bionetgen/bng-linux:')
elif platform.system() == 'Darwin':
    BioNetGen=os.path.join(distpath, 'bionetgen/bng-mac:')
elif platform.system()=='Windows':
    BioNetGen=os.path.join(distpath, 'bionetgen/bng-win:')
else:
    raise ValueError('BioNetGen error. Platform unknown!')
os.environ['PATH']=BioNetGen+os.environ['PATH']

## Defining the PySB model

In [ ]:
# Start model
model = Model()

# Setting Monomers
Monomer('DA', ['DA_b1'])
Monomer('D1R', ['D1R_b1', 'D1R_p', 'D1R_s'], {'D1R_p':['p0','p1'], 'D1R_s':['i','a']})    # Dopamine receptor 1 with two binding sites: for DA and G-proteins
Monomer('Golf', ['Golf_b1'])                                                              # Golf protein (alpha and beta/gamma complex)
Monomer('Gi', ['Gi_b1'])
Monomer('Gbgolf')
Monomer('GaolfGTP', ['GaolfGTP_b1'])
Monomer('GaolfGDP')

# Setting parameters
Parameter('kD1R_DA_1', 0.005)       # 1/(nM*s) |Association constant of the complex D1R_DA
Parameter('kD1R_DA_2', 5.0)         # 1/s      |Dissociation constant of the complex D1R_DA
Parameter('kD1R_DA_Golf_1', 0.003)  # 1/(nM*s) |Association constant of the complex D1R_Golf
Parameter('kD1R_DA_Golf_2', 5.0)    # 1/s      |Dissociation constant of the complex D1R_Golf
Parameter('kD1R_DA_Golf_decay', 15) # 1/s      |Rate of formation of
Parameter('kGaolfGTP_decay', 30)    # 1/s      |Rate of convertion of GaolfGTP into GaolfGDP
Parameter('kGolf_formation', 100)   # 1/s      |Rate of formation of Golf


# Setting initials

Initial(DA(DA_b1=None), Parameter('DA_init', 2000))
Initial(D1R(D1R_b1=None, D1R_p='p0', D1R_s='i'), Parameter('D1R_0', 2000.0))   
Initial(Golf(Golf_b1=None), Parameter('Golf_0', 2000.0)) 

# Setting reactions
##Dopamine and the G-protein Coupled RECEPTORS
'''The D1R can bind either the inactive G protein first, and then
dopamine, or dopamine first and then and then the inactivate G protein.'''
Rule('reaction1', D1R(D1R_b1=None, D1R_p='p0', D1R_s='i') + DA(DA_b1=None) | D1R(D1R_b1=None, D1R_p='p0', D1R_s='a'), kD1R_DA_1, kD1R_DA_2)
Rule('reaction3', D1R(D1R_b1=None, D1R_p='p0', D1R_s='a') + Golf(Golf_b1=None) | D1R(D1R_b1=50, D1R_p='p0', D1R_s='a') % Golf(Golf_b1=50), kD1R_DA_Golf_1, kD1R_DA_Golf_2)
Rule('reaction9', D1R(D1R_b1=50, D1R_p='p0', D1R_s='a') % Golf(Golf_b1=50) >> D1R(D1R_b1=None, D1R_p='p0', D1R_s='a') + Gbgolf() + GaolfGTP(GaolfGTP_b1=None) , kD1R_DA_Golf_decay)
Rule('reaction10', GaolfGTP(GaolfGTP_b1=None) >> GaolfGDP(), kGaolfGTP_decay)
Rule('reaction11', GaolfGDP() + Gbgolf() >> Golf(Golf_b1=None), kGolf_formation)

# Setting Observables
Observable('obs_DA',DA(DA_b1=None))
Observable('obs_D1R',D1R(D1R_b1=None, D1R_p='p0', D1R_s='i'))
Observable('obs_D1RDA',D1R(D1R_b1=None, D1R_p='p0', D1R_s='a'))
Observable('obs_Golf',Golf(Golf_b1=None))
Observable('obs_Gbgolf',Gbgolf())
Observable('obs_GaolfGTP',GaolfGTP(GaolfGTP_b1=None))
Observable('obs_GaolfGDP',GaolfGDP());




## Setting simulation parameters


In [ ]:
ttime = 1     # time in seconds
nsteps = 1000 # time step

t = np.geomspace(0.00001, ttime, nsteps)


## Run Simulation

In [ ]:
simres = ScipyOdeSimulator(model, tspan=t, compiler='python').run()
yout = simres.all

## Vizualize results

In [ ]:
# Create a figure for the plot
fig, ax = plt.subplots(figsize=[10, 6])

# Get data
X = t.copy()
Y = yout['obs_D1R']
Y2= yout['obs_D1RDA']

# Plot data
plt.plot(X, Y, label='D1R')
plt.plot(X, Y2, label='D1RDA')

# Setting axis
ax.set_xlabel('Autodock binding score')
ax.set_ylabel('Count')

# Display the legend
plt.legend()

# Show the plot
plt.show()